In [2]:
import pandas as pd
import numpy as np
from norn.poems import Poem, PoemCollection, is_valid_urn

In [14]:
data = "/home/larsm/projects/NORN/Extract_poems/1890 enkeltdikt.xlsx"

df = pd.read_excel(data, header=None)

errors = pd.read_csv("error_urn.csv", header=0)

In [5]:
# Create list of books

book_list = []
book = None
for i, row in df.iterrows():
    
    #if row[0] is not np.nan:
    if is_valid_urn(str(row[0])):
        if book is not None:
            book_list.append(book)
        book = PoemCollection([], *row[:6])
    else:
        if row[1] == "Tittel på dikt":
            continue
        elif row[1] is np.nan:
            continue
        else:
            book.poems.append(Poem(book.urn, *row[1:7]))
            
poem_list = [x.poems for x in book_list if len(x.poems) > 0] # Remove books without poems added
poems = [x for sublist in poem_list for x in sublist] # Flatten list of lists

In [6]:
source_poems = pd.DataFrame(poems)

In [7]:
import os
import pandas as pd

def read_file_contents(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def create_dataframe_from_folder(main_folder):
    data = []

    for root, dirs, files in os.walk(main_folder):
        for file in files:
            if file.endswith('.txt'):
                subfolder = os.path.basename(root)
                file_path = os.path.join(root, file)
                contents = read_file_contents(file_path)
                data.append([subfolder, os.path.splitext(file)[0], contents])

    return pd.DataFrame(data, columns=['urn', 'poem', 'text'])


In [8]:
text_df = create_dataframe_from_folder("/home/larsm/projects/NORN/Extract_poems/texts_joined")

In [9]:
text_df

,urn,poem,text
0,URN:NBN:no-nb_digibok_2008040204092,Mitt beste kvad,Mitt beste kvad. \nF \nUn syngja eg kunde \nei...
1,URN:NBN:no-nb_digibok_2008040204092,Paa villan fot,Paa villan fot. \n» \nE: stengjer mine dørar \...
2,URN:NBN:no-nb_digibok_2008040204092,Aashild jorda,"Åashild jorda. \n[f \n\n\nAr jord er du komi, ..."
3,URN:NBN:no-nb_digibok_2008040204092,Heime på Vegard,Heime paa Vegard. \n* \n\nE: sit med hagegjerd...
4,URN:NBN:no-nb_digibok_2008040204092,Naudardom,Naudard om. \n\n\n[ \nWimranæla gjeng dei i vi...
...,...,...,...
2457,URN:NBN:no-nb_digibok_2009010803011,Nu vaagner alle smaa vover!,Nu vaagner alle smaa vover! \n\n\nNu vaagner a...
2458,URN:NBN:no-nb_digibok_2009010803011,Regn,"Regn. \n(Impromptu.) \n\n\nEn er en, og to er ..."
2459,URN:NBN:no-nb_digibok_2009010803011,Genre,Genre. \n\n\nDe sover. \n\nBegge sover. \n\nDe...
2460,URN:NBN:no-nb_digibok_2009010803011,Eva,"E va. ; \n\n\nBlygraat er havet, blygraa himle..."


In [17]:
source_poems.urn.nunique(), text_df.urn.nunique(), errors['0'].nunique()

(71, 57, 14)

In [23]:
source_w_o_errors = source_poems.loc[~source_poems.urn.isin(errors['0'])]

In [26]:
source_w_o_errors

,urn,title,page_start,page_end,overlapp,digital_visning,comment
0,URN:NBN:no-nb_digibok_2009032303011,Variation,8,9.0,NaN,NaN,NaN
1,URN:NBN:no-nb_digibok_2009032303011,Pigen med fuglefælden,10,12.0,NaN,NaN,NaN
2,URN:NBN:no-nb_digibok_2009032303011,Troldsøstre,13,14.0,NaN,NaN,NaN
3,URN:NBN:no-nb_digibok_2009032303011,I Sarons dal,15,16.0,NaN,NaN,NaN
4,URN:NBN:no-nb_digibok_2009032303011,Idyl,17,18.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2914,URN:NBN:no-nb_digibok_2013061808054,Erindringer,46,47.0,NaN,NaN,NaN
2915,URN:NBN:no-nb_digibok_2013061808054,Angst,48,48.0,NaN,NaN,NaN
2916,URN:NBN:no-nb_digibok_2013061808054,Forlovet,49,49.0,NaN,NaN,NaN
2917,URN:NBN:no-nb_digibok_2013061808054,Vaagne drømme,50,50.0,NaN,NaN,NaN


In [78]:
source_w_o_errors.loc[:, "title"] = source_w_o_errors.title.astype(str)
missing_poems = source_w_o_errors.loc[~source_w_o_errors.title.isin(text_df.poem.apply(str))]

In [79]:
errors

,0
0,URN:NBN:no-nb_digibok_2011031412004
1,URN:NBN:no-nb_digibok_2018020748046
2,URN:NBN:no-nb_digibok_2018042548139
3,URN:NBN:no-nb_digibok_2017031548123
4,URN:NBN:no-nb_digibok_2017080748090
5,URN:NBN:no-nb_digibok_2020110907594
6,URN:NBN:no-nb_digibok_2017060248006
7,URN:NBN:no-nb_digibok_2009040303003
8,URN:NBN:no-nb_digibok_2021041348661
9,URN:NBN:no-nb_digibok_2018042748096


In [80]:
missing_poems.urn.value_counts()

urn
URN:NBN:no-nb_digibok_2009032303010    1
Name: count, dtype: int64